In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
glanceData = pd.read_csv("/content/drive/MyDrive/Glance_Views.csv")
salesData = pd.read_csv("/content/drive/MyDrive/Sales_Data.csv")
mergedData = pd.merge(salesData, glanceData, on=['SKU_NAME', 'FEED_DATE'])

In [ ]:
#Q1 - Identify the most expensive SKU, on average, over the entire time period.
averagePrice = salesData.groupby('SKU_NAME')['ORDERED_REVENUE'].mean().sort_values(ascending=False)
mostExp = averagePrice.idxmax()
print("Most expensive SKU, on average, over the entire time period:", mostExp)

In [ ]:
#Q2 What % of SKUs have generated some revenue in this time period?
revenuePer = (salesData['ORDERED_REVENUE'].gt(0).sum() / salesData['ORDERED_REVENUE'].count()) * 100
print("Percentage of SKUs that have generated revenue in this time period:", revenuePer)

In [ ]:
# Q3 - Somewhere in this timeframe, there was a Sale Event. Identify the dates.

#                                K-Means Clustering
scaler = StandardScaler()
scaledData = scaler.fit_transform(salesData[['ORDERED_REVENUE']])
kmeans = KMeans(n_clusters=2, n_init=10, random_state=0).fit(scaledData)

salesData['Cluster'] = kmeans.labels_

sale_event_dates = salesData[salesData['Cluster'] == 1]['FEED_DATE']
print(sale_event_dates)

In [ ]:
#Q4 (Dependent on 3) Does having a sale event cannibalize sales in the immediate
# aftermath? Highlighting a few examples would suffice (brownie points - determine a
# statistical metric to prove/disprove this).
plt.figure(figsize=(12, 6))
sns.lineplot(data=salesData, x='FEED_DATE', y='ORDERED_REVENUE')
plt.title("Ordered Revenue Over Time")
plt.xlabel("Date")
plt.ylabel("Ordered Revenue")
for sale_event_date in sale_event_dates:
    plt.axvline(x=sale_event_date, color='r', linestyle='--')
plt.show()

In [ ]:
# Q5 In each category, find the subcategory that has grown slowest
# relative to the category it is present in. If you were handling the entire
# portfolio, which of these subcategories would you be most concerned
# with?
categoryGrowth = salesData.groupby(['CATEGORY', 'SUB_CATEGORY']).agg({'ORDERED_REVENUE': 'sum'}).groupby(level=0).apply(lambda x: x / float(x.sum())).reset_index()
slowest_growing_subcategories = categoryGrowth.loc[categoryGrowth.groupby('CATEGORY')['ORDERED_REVENUE'].idxmin()]

plt.figure(figsize=(12, 6))
sns.barplot(data=slowest_growing_subcategories, x='CATEGORY', y='ORDERED_REVENUE', hue='SUB_CATEGORY')
plt.title("Slowest-growing Subcategory in Each Category")
plt.xlabel("Category")
plt.ylabel("Relative Growth Rate")
plt.legend(title="Subcategory", bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Q6 Highlight any anomalies/mismatches in the data that you see, if any 
# (In terms of data quality issues)Highlight any anomalies/mismatches in the data that you see, if any.
# (In terms of data quality issues)

pca = PCA(n_components=2)
pca = PCA(n_components=1)
principalComponents = pca.fit_transform(salesData[['ORDERED_REVENUE']])
principalDf = pd.DataFrame(data = principalComponents, columns = ['principal component 1'])
plt.figure(figsize=(12, 6))
sns.boxplot(data=principalDf)
plt.title("Distribution of Ordered Revenue")
plt.xlabel("Ordered Revenue")
plt.show()

In [ ]:
# Q7 For SKU Name C120[H:8NV, discuss whether Unit Conversion
# (Units/Views) is affected by Average Selling Price. (brownie points -
# determine a statistical technique to test this)
plt.figure(figsize=(10, 8))
sns.set_style("whitegrid")
sns.jointplot(data=mergedData, x='ORDERED_REVENUE', y='UNITS', kind='reg', scatter_kws={'alpha':0.5})
plt.title("Average Selling Price vs. Unit Conversion Rate", pad=100)
plt.xlabel("Average Selling Price")
plt.ylabel("Unit Conversion Rate")
plt.show()